In [1]:
%matplotlib widget
# %matplotlib qt
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
import sympy as sp
from sympy import symbols
import bmcs_utils.api as bu
from bmcs_cross_section.api import MKappa, TShape, EC2, BarLayer, ReinfLayer
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, BeamSLSCurve, LoadDeflectionParamsStudy
from os.path import join
from mpl_toolkits import mplot3d
sp.init_printing()

In [2]:
def get_Fu_and_Fs(dp, n_i, rho_range=(0.0025, 0.025), slenderness_range=(10, 35), upper_reinforcement = False):
    rho_range = np.linspace(*rho_range, n_i)
    slenderness_range = np.linspace(*slenderness_range, n_i)

# def get_Fu_and_Fs(dp, rho_range=np.linspace(0.0025, 0.025), slenderness_range=np.linspace(10, 35), upper_reinforcement = False):
    if upper_reinforcement:
        d = dp.mc.cross_section_layout.items[0].z
    else:
        d = dp.mc.cross_section_shape_.H -  dp.mc.cross_section_layout.items[0].z
    b = dp.mc.cross_section_shape_.B
    area_g = b * d

    rho_grid, sl_grid = np.meshgrid(rho_range, slenderness_range) 
    F_u_grid = np.zeros_like(rho_grid)
    F_s_grid = np.zeros_like(rho_grid)

    _ , ax = plt.subplots()
    ax.set_xlabel(r'$w$ [mm]')
    ax.set_ylabel(r'$F$ [KN]')

    for sl_idx in range(0,len(slenderness_range)):
        for rho_idx in range(0,len(rho_range)):      
            rho = rho_grid[rho_idx, sl_idx]
            sl = sl_grid[rho_idx, sl_idx]

            print('parameter combination', rho, sl)

            # assigning the grid area (area_g) to the reinforcement area variable
            A_j_g = rho * area_g
            dp.mc.cross_section_layout.items[0].A = A_j_g

            # assigning the grid length (L_g) to the beam length variable
            L_g = sl * d
            dp.beam_design.L = L_g
            
            dp.mc.state_changed = True

            # running the deflection analysis 
            F_data, w_data = dp.get_Fw()

            # plotting, post-processing & saving the data
            ax.plot(w_data, F_data/1000, label = "rho={}%-sl={} ".format(rho * 100, sl))

            w_s = dp.beam_design.L / 250
            F_u = max(F_data)
            F_s = np.interp(w_s, w_data, F_data, right=F_u*2)

            F_u_grid[rho_idx,sl_idx] = F_u
            F_s_grid[rho_idx,sl_idx] = F_s
            
    return F_u_grid, F_s_grid, rho_grid, sl_grid

### 1. EC2 - Steel

In [3]:
# Concrete C30/37

f_ck = 30

b = 1000
h = 300
d = 0.9 * h

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - h /1000) * f_ctd, f_ctd)

f_ctm = EC2.get_f_ctm(f_ck)
print(f_ctm)
# Info: eps_cr =  0.000170 with default concrete law gives good slenderness curve

mc1 = MKappa(low_kappa = 0, high_kappa = 0.00003)

mc1.cs_design.matrix='EC2 with plateau'
mc1.cs_design.matrix_.trait_set(
#         eps_cr=2.5* f_ctd_fl /E, #0.000125 * f_ck/30, # f_ctm /E,
#         eps_tu=0.002,
#         mu=0.0,
    f_cm = EC2.get_f_cm(f_ck),
    factor = 0.85/1.5,
    )

mc1.cross_section_shape_.B = b
mc1.cross_section_shape_.H = h

rho = 0.01
A_s= rho * b * d

# NOTE THAT FOR STEEL STRENGTH IS CALLED f_sy

bl1 = ReinfLayer(name='layer1', z=h-d, A=A_s, matmod='steel')
bl1.matmod_.trait_set(E_s=200000, f_sy=500, factor = 1/1.15 )
mc1.cross_section_layout.add_layer(bl1)

dp1 = DeflectionProfile(mc=mc1)
dp1.beam_design.beam_conf_name = BoundaryConfig.SIMPLE_BEAM_DIST_LOAD
# dp1.beam_design.beam_conf_name = BoundaryConfig.FIXED_AND_ROLLER_SUPPORT_DIST_LOAD
# dp1.beam_design.beam_conf_name = BoundaryConfig.FIXED_SUPPORT_DIST_LOAD
# dp1.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD

dp1.F_scale = 1
dp1.beam_design.L = 8 * d # l/d = 10

2.896468153816889


In [4]:
dp1.interact()

In [5]:
F_u_grid_s, F_s_grid_s, rho_grid_s, sl_grid_s = get_Fu_and_Fs(dp1, 6)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

parameter combination 0.0025 10.0
parameter combination 0.0025 15.0
parameter combination 0.0025 20.0
parameter combination 0.0025 25.0
parameter combination 0.0025 30.0
parameter combination 0.0025 35.0
parameter combination 0.007000000000000001 10.0
parameter combination 0.007000000000000001 15.0
parameter combination 0.007000000000000001 20.0
parameter combination 0.007000000000000001 25.0
parameter combination 0.007000000000000001 30.0
parameter combination 0.007000000000000001 35.0
parameter combination 0.011500000000000002 10.0
parameter combination 0.011500000000000002 15.0
parameter combination 0.011500000000000002 20.0
parameter combination 0.011500000000000002 25.0
parameter combination 0.011500000000000002 30.0
parameter combination 0.011500000000000002 35.0
parameter combination 0.016 10.0
parameter combination 0.016 15.0
parameter combination 0.016 20.0
parameter combination 0.016 25.0
parameter combination 0.016 30.0
parameter combination 0.016 35.0
parameter combination 

In [10]:
np.save('F_u_grid_steel.npy', F_u_grid_s)
np.save('F_s_grid_steel.npy', F_s_grid_s)
np.save('rho_grid_steel.npy', rho_grid_s)
np.save('sl_grid_steel.npy', sl_grid_s)

### 2. EC2 - Carbon

In [3]:
f_ck = 30

b = 1000
h = 300
d = 0.9 * h

E = EC2.get_E_cm(f_ck)

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - h /1000) * f_ctd, f_ctd)

mc2 = MKappa(low_kappa = 0, 
            high_kappa = 0.00007,
             n_kappa= 200,
           ) 

mc2.cs_design.matrix='EC2 with plateau'
mc2.cs_design.matrix_.trait_set(
        eps_cr=f_ctd_fl /E,
    f_cm = EC2.get_f_cm(f_ck),
    factor = 0.85/1.5
    )  

mc2.cross_section_shape_.B = b
mc2.cross_section_shape_.H = h

rho = 0.01
# A_s=6 * np.pi * (32/2)**2
A_s= rho * b * d

bl2 = ReinfLayer(name='layer1', z=h-d, A=A_s, matmod='carbon')
bl2.matmod_.trait_set(E=230000, f_t=3000, factor = 1./1.5)
mc2.cross_section_layout.add_layer(bl2)

dp2 = DeflectionProfile(mc=mc2)
dp2.beam_design.beam_conf_name = BoundaryConfig.SIMPLE_BEAM_DIST_LOAD

dp2.F_scale = 1
dp2.beam_design.L = 8 * d

# rho1 = np.linspace(0.0005, 0.007, 7)
# rho2 = np.linspace(0.0075, 0.025, 5)
# rho_range=np.concatenate((rho1, rho2))

# sl1 = np.linspace(3, 10, 7)
# sl2 = np.linspace(11, 50, 5)
# slenderness_range=np.concatenate((sl1, sl2))

In [4]:
dp2.interact()

In [5]:
for f_ck in [30, 50, 70, 90, 100]:

    b = 1000
    h = 300
    d = 0.9 * h

    E = EC2.get_E_cm(f_ck)

    fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
    f_ctd = fctk_0_05
    # f_ctd = 0.85 * fctk_0_05 / 1.5
    f_ctd_fl = max((1.6 - h /1000) * f_ctd, f_ctd)

    eps_cr = f_ctd_fl/E

    mc2 = MKappa(low_kappa = 0, 
                high_kappa = 0.00007, # 0.00005,
               ) 
    mc2.cs_design.matrix='EC2 with plateau'
    mc2.cs_design.matrix_.trait_set(
            E_cc=E,
            E_ct=E,
            eps_cr=f_ctd_fl /E, #eps_cr, #0.000132 * f_ck/30,
            eps_cy=EC2.get_eps_c2(f_ck),
            eps_cu=EC2.get_eps_cu2(f_ck),
            eps_tu=0.002,
            mu=0.0,
            f_cd = f_ck*0.85/1.5,
    #     factor = 1
    #     factor = 0.85/1.5
        )  

    mc2.cross_section_shape_.B = b
    mc2.cross_section_shape_.H = h

    rho = 0.01
    # A_s=6 * np.pi * (32/2)**2
    A_s= rho * b * d

    bl2 = ReinfLayer(name='layer1', z=h-d, A=A_s, matmod='carbon')
    bl2.matmod_.trait_set(E=230000, f_t=3000, factor = 1./1.5) #factor = 1.)
    mc2.cross_section_layout.add_layer(bl2)

    dp2 = DeflectionProfile(mc=mc2)
    dp2.beam_design.beam_conf_name = BoundaryConfig.SIMPLE_BEAM_DIST_LOAD

    dp2.F_scale = 1
    dp2.beam_design.L = 10 * d

    rho1 = np.linspace(0.0005, 0.007, 12)
    rho2 = np.linspace(0.0075, 0.025, 8)
    rho_range=np.concatenate((rho1, rho2))

    sl1 = np.linspace(3, 10, 12)
    sl2 = np.linspace(11, 50, 8)
    slenderness_range=np.concatenate((sl1, sl2))
    
    F_u_grid_c, F_s_grid_c, rho_grid_c, sl_grid_c = get_Fu_and_Fs(dp2, rho_range=rho_range, slenderness_range=slenderness_range)
    
    np.save('F_u_grid_carbon_c' + str(f_ck) + '.npy', F_u_grid_c)
    np.save('F_s_grid_carbon_c' + str(f_ck) + '.npy', F_s_grid_c)
    np.save('rho_grid_carbon_c' + str(f_ck) + '.npy', rho_grid_c)
    np.save('sl_grid_carbon_c' + str(f_ck) + '.npy', sl_grid_c)


TraitError: The 'eps_cy' trait of an EC2PlateauConcreteMatMod instance is 'read only'.

In [7]:
dp2.interact()

In [5]:
F_u_grid_c, F_s_grid_c, rho_grid_c, sl_grid_c = get_Fu_and_Fs(dp2, 7, rho_range=(0.0002, 0.025), slenderness_range=(3, 50))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

parameter combination 0.0002 3.0
parameter combination 0.0002 10.833333333333332
parameter combination 0.0002 18.666666666666664
parameter combination 0.0002 26.5
parameter combination 0.0002 34.33333333333333
parameter combination 0.0002 42.166666666666664
parameter combination 0.0002 50.0
parameter combination 0.004333333333333333 3.0
parameter combination 0.004333333333333333 10.833333333333332
parameter combination 0.004333333333333333 18.666666666666664
parameter combination 0.004333333333333333 26.5
parameter combination 0.004333333333333333 34.33333333333333
parameter combination 0.004333333333333333 42.166666666666664
parameter combination 0.004333333333333333 50.0
parameter combination 0.008466666666666667 3.0
parameter combination 0.008466666666666667 10.833333333333332
parameter combination 0.008466666666666667 18.666666666666664
parameter combination 0.008466666666666667 26.5
parameter combination 0.008466666666666667 34.33333333333333
parameter combination 0.00846666666666

In [9]:
np.save('F_u_grid_carbon.npy', F_u_grid_c)
np.save('F_s_grid_carbon.npy', F_s_grid_c)
np.save('rho_grid_carbon.npy', rho_grid_c)
np.save('sl_grid_carbon.npy', sl_grid_c)

In [6]:
fig, ax = plt.subplots()

z = F_u_grid_s / F_s_grid_s - 1./0.59
CS = ax.contour(rho_grid_s, sl_grid_s, z, levels=[0])
ax.clabel(CS, inline=1, fontsize=10)

# z2 = F_u_grid_c / F_s_grid_c - 1./0.59
# CS2 = ax.contour(rho_grid_c, sl_grid_c, z2)
# ax.clabel(CS2, inline=1, fontsize=10)


ax.set_ylabel('L/d')
ax.set_xlabel(r'$\rho$ %')
ax.set_ylim(0, 35)
ax.set_xlim(0.0, 0.025)
ax.plot([0.005, 0.005],[0, 35], c='black', lw=0.8)
# Draw EC2 curve
BeamSLSCurve().plot_steel_sls_curves(ax=ax, f_cks=[30], axes_start_from_zero=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Checking steel and carbon graphs for reinforcement ratio rho = 0.5%

## Steel

In [5]:
# set rho = 0.5%
d = 0.9 * mc1.cross_section_shape_.H 
mc1.cross_section_layout.items[0].A = 0.005 * mc1.cross_section_shape_.B * d
mc1.state_changed = True

params_config = [
    ['L',   dp1.beam_design, [7*d, 20*d]],
]

LoadDeflectionParamsStudy(dp1, show_sls_deflection_limit = True).run(params_config)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Parametric study is running...
L: (1890.0), (5400.0), 
Parametric study finished.


In [3]:
mc1.cross_section_layout.items[0].A = 0.005 * mc1.cross_section_shape_.B * (0.9 * mc1.cross_section_shape_.H)
mc1.state_changed = True
dp1.beam_design.L = 7 * d

print('L = ' + str(dp1.beam_design.L/1000) + ' m')
print('w_limit = ' + str(dp1.beam_design.L/250) + ' mm')
dp1.interact()

L = 1.89 m
w_limit = 7.56 mm


## Carbon

In [6]:
# set rho = 0.5%
d = 0.9 * mc2.cross_section_shape_.H 
mc2.cross_section_layout.items[0].A = 0.005 * mc2.cross_section_shape_.B * d
mc2.state_changed = True

params_config = [
    ['L',   dp2.beam_design, [7*d, 20*d]],
#         ['L',   dp2.beam_design, [5*d, 7*d, 10*d, 15*d, 20*d, 25*d]],
]

LoadDeflectionParamsStudy(dp2, show_sls_deflection_limit = True).run(params_config)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Parametric study is running...
L: (1890.0), (5400.0), 
Parametric study finished.


In [16]:
mc2.cross_section_layout.items[0].A = 0.005* mc2.cross_section_shape_.B * (0.9 * mc2.cross_section_shape_.H)
mc2.state_changed = True
dp2.beam_design.L = 30 * d
dp2.interact()

## Plot all curves

In [16]:
f_cks = [50]
F_u_grids = []
F_s_grids = []
rho_grids = []
sl_grids = []
reinforcement = 'carbon'
for f_ck in f_cks:
    f_ck = str(f_ck)

    F_u_grids.append(np.load('exported_data/F_u_grid_carbon_EC2_eq2_tension_E230_ft_3000_c' + str(f_ck) + '.npy'))
    F_s_grids.append(np.load('exported_data/F_s_grid_carbon_EC2_eq2_tension_E230_ft_3000_c' + str(f_ck) + '.npy'))
    rho_grids.append(np.load('exported_data/rho_grid_carbon_EC2_eq2_tension_E230_ft_3000_c' + str(f_ck) + '.npy'))
    sl_grids.append(np.load('exported_data/sl_grid_carbon_EC2_eq2_tension_E230_ft_3000_c' + str(f_ck) + '.npy'))

In [22]:
_, ax = plt.subplots(1,1)

ax.set_ylabel('L/d')
ax.set_xlabel(r'$\rho$ %')
ax.set_ylim(0, 35)
ax.set_xlim(0.0, 0.025)

for f_ck, F_u_grid, F_s_grid, rho_grid, sl_grid in zip(f_cks, F_u_grids, F_s_grids, rho_grids, sl_grids):
    z = 0.5 * F_u_grid / F_s_grid - 1./0.59
    CS = ax.contour(rho_grid, sl_grid, z, colors=[np.random.rand(3,)], levels=[0])
    CS.collections[0].set_label('C' + str(f_ck))
#     ax.clabel(CS, inline=1, fontsize=10)
    
    BeamSLSCurve().plot_steel_sls_curves(ax=ax, f_cks=[50], axes_start_from_zero=True)

ax.legend()
